In [5]:
import numpy as np
import matplotlib.pyplot as plt
import math
from tqdm import tqdm
import torch
import torch.nn as nn

In [6]:
class NN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super(NN, self).__init__(*args, **kwargs)
        self.net = torch.nn.Sequential(
            nn.Linear(3, 16),
            nn.Tanh(),
            nn.Linear(16, 32),
            nn.Tanh(),
            nn.Linear(32, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 32),
            nn.Tanh(),
            nn.Linear(32,16),
            nn.Tanh(),
            nn.Linear(16, 16),
            nn.Tanh(),
            nn.Linear(16, 2),
        )

    def forward(self, x):
        out = self.net(x)
        return out

In [ ]:
def boundary_condition(x, custom_value):
    return torch.full_like(x, custom_value)

In [18]:
num_points = 10

In [24]:
x_boundary = torch.tensor([0.0, 1.0])
x_boundary = x_boundary.repeat(num_points // 2)
x_boundary

tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])

In [21]:
y_boundary = torch.rand(num_points)
y_boundary

tensor([0.7925, 0.4851, 0.2120, 0.7314, 0.0318, 0.2770, 0.8334, 0.9435, 0.0896,
        0.7129])

In [25]:
if torch.rand(1) > 0.5:
    x_boundary, y_boundary = y_boundary, x_boundary

In [26]:
x_boundary

tensor([0.7925, 0.4851, 0.2120, 0.7314, 0.0318, 0.2770, 0.8334, 0.9435, 0.0896,
        0.7129])

In [27]:
y_boundary

tensor([0., 1., 0., 1., 0., 1., 0., 1., 0., 1.])

In [15]:
x_boundary = torch.tensor([0.0, 1.0]).repeat(num_points // 2)
y_boundary = torch.rand(num_points)

if torch.rand(1) > 0.5:
    x_boundary, y_boundary = y_boundary, x_boundary

return x_boundary.view(-1, 1), y_boundary.view(-1, 1)


tensor([0.6978])

In [3]:
h = 0.01
k = 0.01
Db = 0.0001
Dn = 0.0001
phi = 0.2
ksi = 0.0
cb = 0.154
lambd_nb = 40
mi_n = 0.2
lambd_bn = 0.1
y_n = 0.1
x_dom = (0,1)
y_dom = (0,1)
t_dom = (0,10)

size_x = int(((x_dom[1] - x_dom[0]) / (h)) + 1)
size_y = int(((y_dom[1] - y_dom[0]) / (h)) + 1)
size_tt = int(((t_dom[1] - t_dom[0]) / (k)) + 1)

In [4]:
x = np.linspace((x_dom[1], x_dom[0]) , size_x)
y = np.linspace((y_dom[1], y_dom[0]) , size_y)
t = np.linspace((t_dom[1], t_dom[0]) , size_tt)
